# [03] COVID Image Classification: MobileNet 구조 만들기

본 실습에서는 이미지 분류에서 경량화 모델로 널리 알려진 MobileNet V2를 구현해보겠습니다.

논문의 링크는 이곳입니다. (https://arxiv.org/pdf/1801.04381.pdf)

구조에 대해서 간단하게 짚어보겠습니다. 아래의 그림들은 모두 위의 논문에서 발췌하였습니다.


![MobileNetV2](./imgs/mobileNetV2.png)

경량화 신경망들의 구조를 뽑아본 것입니다. 여기서 가장 눈여겨 볼 것은 MobileNet과 MobileNetV2입니다.

두 신경망의 가장 큰 차이는 `bottleneck` 구조입니다. `Inverted Residual Block`으로 보통 부릅니다.

아래는 `MobileNet`에 대한 설명입니다.

![overview_mobilenet](./imgs/overview_mobilenet.png)

반면, MobileNetV2는 기존의 구조에 bottleneck 구조를 더한 아래와 같은 형태입니다.

- `1x1` => `3x3` => `1x1`을 이어서 block을 형성한 것입니다.

![bottleneck](./imgs/bottleneck.png)

이것을 기반으로 MobileNetV2는 아래처럼 구조를 만들었습니다.

![overview](./imgs/overview_mobilenetv2.png)

아래의 코드는 파이토치의 official 코드를 조금 고친 것입니다.

In [1]:
import torch
from torch import nn

In [4]:
class ConvBNReLU(nn.Module):
    def __init__(self, inplanes, outplanes, kernel_size, stride, groups):
        super(ConvBNReLU, self).__init__()
        padding = (kernel_size - 1) // 2
        self.conv = None
        self.bn = nn.BatchNorm2d()
        self.act = None
        
    def forward(self,x):
        
        x1 = self.conv(x)
        x1 = self.bn(x1)
        x1 = self.act(x1)
        
        return x1

## InvertedResidual Block Remind 요약
- 기존의 Convolution filter를 두 가지로 분리해서 붙이는 것을 고안한 방법입니다..
    1. Depthwise convolution: 3x3 convolution이나 channel마다 계산을 하는 (평면적으로) 함수.
    2. Pointwise convolution: 1x1의 original convolution의 형태. 서로 다른 channel들의 값을 고려하는 함수.
- 위의 내용에 더해 expansion - convolution - squeeze의 형태로 block이 구성됩니다. (Inverted Residual Block)
- 쉽게 이야기하면 아래와 같습니다.
    1. 처음 들어온 input의 channel을 확장시키는 pointwise convolution을 통과함.
    2. channel의 수가 늘어난 input을 depthwise convolution을 통과함.
    3. 다시 channel 수를 줄이는 (squeeze) pointwise convolution을 통과함.

In [2]:
class InvertedResidual(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio, shortcut):
        super(InvertedResidual, self).__init__()
        
        self.shortcut = shortcut # True, False
        
        hidden_dim = int(round(inp * expand_ratio))
        
        self.expand_pw = ConvBNReLU()
        self.depthwise = ConvBNReLU()
        self.squeeze_pw = ConvBNReLU()
        
    def forward(self, x):
        x1 = self.expand_pw(x)
        x1 = self.depthwise(x)
        x1 = self.squeeze_pw(x)
        
        if self.shortcut:
            out = x + x1
        else:
            out = x1
            
        return out

In [ ]:
class LGMobileNetV2(nn.Module):
    def __init__(self, num_classes):
        super(LGMobileNetV2, self).__init__()
        
        # input : 3 x 150 x 150
        self.stem = ConvBNReLU(inplanes=3, outplanes=16, kernel_size=3, stride=2) # 16 x 75 x 75
        self.block1 = InvertedResidual(inp, oup, stride, expand_ratio, shortcut) # 16 x 75 x 75
        self.block2 = InvertedResidual(inp, oup, stride, expand_ratio, shortcut) # 32 x 38 x 38
        self.block3 = InvertedResidual(inp, oup, stride, expand_ratio, shortcut) # 64 x 19 x 19
        self.block4 = InvertedResidual(inp, oup, stride, expand_ratio, shortcut) # 64 x 19 x 19
        self.block5 = InvertedResidual(inp, oup, stride, expand_ratio, shortcut) # 128 x 10 x 10
        self.block6 = InvertedResidual(inp, oup, stride, expand_ratio, shortcut) # 128 x 5 x 5
        self.block7 = InvertedResidual(inp, oup, stride, expand_ratio, shortcut) # 128 x 5 x 5
        self.block8 = InvertedResidual(inp, oup, stride, expand_ratio, shortcut) # 128 x 3 x 3
        self.pool = nn.AvgPool2d(kernel_size=3)
        
        self.classifier = nn.Linear(128,2)
        
    def forward(self, x):
        x1 = self.stem(x)
        
        x1 = self.block1(x1)
        x1 = self.block2(x1)
        x1 = self.block3(x1)
        x1 = self.block4(x1)
        x1 = self.block5(x1)
        x1 = self.block6(x1)
        x1 = self.block7(x1)
        x1 = self.block8(x1)
        
        x1 = x1.reshape(x.shape[0], -1) # 2차원으로 바꾸는 코드
        x1 = self.classifier(x1)
        
        return x1

---